# APIM ❤️ OpenAI

## Backend pool Load Balancing lab
![flow](../../images/backend-pool-load-balancing.gif)

Playground to try the built-in load balancing [backend pool functionality of APIM](https://learn.microsoft.com/azure/api-management/backends?tabs=bicep) to either a list of Azure OpenAI endpoints.

Notes:
- The backend pool uses round-robin by default
- Priority and weight-based routing are also supported: Adjust the `priority` (the lower the number, the higher the priority) and `weight` parameters in the `openai_resources` variable
- The `retry` API Management policy initiates a retry to an available backend if an HTTP 429 status code is encountered

### Result
![result](result.png)

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [🧪 Test the API using a direct HTTP call](#requests)
- [🔍 Analyze Load Balancing results](#plot)
- [🧪 Test the API using the Azure OpenAI Python SDK](#sdk)
- [🗑️ Clean up resources](#clean)

### Prerequisites
- [Python 3.9 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [1]:
import os

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

openai_resources = [
    {"name": "openai1", "location": "uksouth", "priority": 1, "weight": 80},
    {"name": "openai2", "location": "swedencentral", "priority": 1, "weight": 10},
    {"name": "openai3", "location": "francecentral", "priority": 1, "weight": 10}
]

openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"


<a id='1'></a>
### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
# %load ../../shared/snippets/create-az-resource-group.py

# type: ignore

import datetime

resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}

if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print(f"✅ Azure Resource Group {resource_group_name} created ⌚ {datetime.datetime.now().time()}")


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

`openAIModelCapacity` is set intentionally low to `8` (8k tokens per minute) in _main.bicep_ to showcase the retry logic in the load balancer.

In [ ]:
# %load ../../shared/snippets/create-az-deployment.py

# type: ignore

import json

backend_id = "openai-backend-pool" if len(openai_resources) > 1 else openai_resources[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_xml = policy_xml_file.read()

    if "{backend-id}" in policy_xml:
        policy_xml = policy_xml.replace("{backend-id}", backend_id)

    if "{aad-client-application-id}" in policy_xml:
        policy_xml = policy_xml.replace("{aad-client-application-id}", client_id)

    if "{aad-tenant-id}" in policy_xml:
        policy_xml = policy_xml.replace("{aad-tenant-id}", tenant_id)

    policy_xml_file.close()
open("policy-updated.xml", 'w').write(policy_xml)

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIAPIVersion": { "value": openai_api_version }
    }
}

with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"


<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [1]:
# %load ../../shared/snippets/deployment-outputs.py
# type: ignore

# Obtain all of the outputs from the deployment
stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs -o json
outputs = json.loads(stdout.n)

# Extract the individual properties
apim_service_id = outputs.get('apimServiceId', {}).get('value', '')
apim_subscription_key = outputs.get('apimSubscriptionKey', {}).get('value', '')
apim_subscription1_key = outputs.get('apimSubscription1Key', {}).get('value', '')
apim_subscription2_key = outputs.get('apimSubscription2Key', {}).get('value', '')
apim_subscription3_key = outputs.get('apimSubscription3Key', {}).get('value', '')
apim_resource_gateway_url = outputs.get('apimResourceGatewayURL', {}).get('value', '')
workspace_id = outputs.get('logAnalyticsWorkspaceId', {}).get('value', '')
app_id = outputs.get('applicationInsightsAppId', {}).get('value', '')
function_app_resource_name = outputs.get('functionAppResourceName', {}).get('value', '')
cosmosdb_connection_string = outputs.get('cosmosDBConnectionString', {}).get('value', '')

# Print the extracted properties if they are not empty
if apim_service_id:
    print(f"👉🏻 APIM Service Id: {apim_service_id}")

if apim_subscription_key:
    print(f"👉🏻 APIM Subscription Key (masked): ****{apim_subscription_key[-4:]}")

if apim_subscription1_key:
    print(f"👉🏻 APIM Subscription Key 1 (masked): ****{apim_subscription1_key[-4:]}")

if apim_subscription2_key:
    print(f"👉🏻 APIM Subscription Key 2 (masked): ****{apim_subscription2_key[-4:]}")

if apim_subscription3_key:
    print(f"👉🏻 APIM Subscription Key 3 (masked): ****{apim_subscription3_key[-4:]}")

if apim_resource_gateway_url:
    print(f"👉🏻 APIM API Gateway URL: {apim_resource_gateway_url}")

if workspace_id:
    print(f"👉🏻 Workspace ID: {workspace_id}")

if app_id:
    print(f"👉🏻 App ID: {app_id}")

if function_app_resource_name:
    print(f"👉🏻 Function Name: {function_app_resource_name}")

if cosmosdb_connection_string:
    print(f"👉🏻 Cosmos DB Connection String: {cosmosdb_connection_string}")


<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

You will not see HTTP 429s returned as API Management's `retry` policy will select an available backend. If no backends are viable, an HTTP 503 will be returned.

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior of the backend pool.

In [ ]:
# %load ../../shared/snippets/api-http-requests.py

import json
import requests
import time

runs = 10
sleep_time_ms = 100
url = f"{apim_resource_gateway_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}"
api_runs = []
messages = {"messages": [
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}

# Initialize a session for connection pooling
session = requests.Session()
# Set default headers
session.headers.update({'api-key': apim_subscription_key})

try:
    for i in range(runs):
        print(f"▶️ Run {i+1}/{runs}:")

        start_time = time.time()
        response = session.post(url, json = messages)
        response_time = time.time() - start_time
        print(f"⌚ {response_time:.2f} seconds")

        # Check the response status code and apply formatting
        if 200 <= response.status_code < 300:
            status_code_str = f"\x1b[1;32m{response.status_code} - {response.reason}\x1b[0m" # Bold and green
        elif response.status_code >= 400:
            status_code_str = f"\x1b[1;31m{response.status_code} - {response.reason}\x1b[0m" # Bold and red
        else:
            status_code_str = str(response.status_code)  # No formatting

        # Print the response status with the appropriate formatting
        print(f"Response status: {status_code_str}")
        print(f"Response headers: {json.dumps(dict(response.headers), indent = 4)}")

        if "x-ms-region" in response.headers:
            print(f"x-ms-region: \x1b[1;32m{response.headers.get("x-ms-region")}\x1b[0m") # this header is useful to determine the region of the backend that served the request
            api_runs.append((response_time, response.headers.get("x-ms-region")))

        if (response.status_code == 200):
            data = json.loads(response.text)
            print(f"Token usage: {json.dumps(dict(data.get("usage")), indent = 4)}\n")
            print(f"💬 {data.get("choices")[0].get("message").get("content")}\n")
        else:
            print(response.text)

        time.sleep(sleep_time_ms/1000)
finally:
    # Close the session to release the connection
    session.close()


<a id='plot'></a>
### 🔍 Analyze Load Balancing results


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle as pltRectangle
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = [15, 7]
df = pd.DataFrame(api_runs, columns = ['Response Time', 'Region'])
df['Run'] = range(1, len(df) + 1)

# Define a color map for each region
color_map = {'UK South': 'lightpink', 'France Central': 'lightblue', 'Sweden Central': 'lightyellow', 'Region3': 'red', 'Region4': 'orange'}  # Add more regions and colors as needed

# Plot the dataframe with colored bars
ax = df.plot(kind = 'bar', x = 'Run', y = 'Response Time', color = [color_map.get(region, 'gray') for region in df['Region']], legend = False)

# Add legend
legend_labels = [pltRectangle((0, 0), 1, 1, color = color_map.get(region, 'gray')) for region in df['Region'].unique()]
ax.legend(legend_labels, df['Region'].unique())

plt.title('Load Balancing results')
plt.xlabel('Run #')
plt.ylabel('Response Time')
plt.xticks(rotation = 0)

average = df['Response Time'].mean()
plt.axhline(y = average, color = 'r', linestyle = '--', label = f'Average: {average:.2f}')

plt.show()

<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK

Repeat the same test using the Python SDK to ensure compatibility. Note that we do not know what region served the response; we only see that we obtained a response.

In [ ]:
# %load ../../shared/snippets/openai-api-requests.py

import time
from openai import AzureOpenAI

runs = 10
sleep_time_ms = 100

client = AzureOpenAI(
    azure_endpoint = apim_resource_gateway_url,
    api_key = apim_subscription_key,
    api_version = openai_api_version
)

messages = [
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]

for i in range(runs):
    print(f"▶️ Run {i+1}/{runs}:")

    start_time = time.time()
    response = client.chat.completions.create(model = openai_model_name, messages = messages) # type: ignore
    response_time = time.time() - start_time
    print(f"⌚ {response_time:.2f} seconds")
    if response.usage:
        print(f"Token usage: Total tokens: {response.usage.total_tokens} (Prompt tokens: {response.usage.prompt_tokens} & Completion tokens: {response.usage.completion_tokens})")
    print(f"💬 {response.choices[0].message.content}\n")

    time.sleep(sleep_time_ms/1000)


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.